In [60]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm

In [61]:
df = pd.read_csv('test.csv')
df1 = pd.read_csv('autoru2.csv')
pd.options.display.max_columns = 30
df1 = df1[0:7400]

In [62]:
df.drop(['vehicleConfiguration','description','name', 'brand', 'id'], axis = 1, inplace = True)
df1.drop(['Unnamed: 0', 'image', 'vehicleConfiguration','name', 'brand','availability','url', 'fuelType.1','name1','name2','priceCurrency'], axis = 1, inplace = True)

In [63]:
#from km in mile
df1.drop(df1[df1.kmage.isna() == True].index, axis = 0, inplace = True)
df1['mileage'] = df1.kmage.apply(lambda x: round(float(x[0:-7]+x[-6:-3])*0.62137))
df1.drop(['kmage'], axis = 1, inplace = True)

In [64]:
df['Владение'].fillna('не имел владельцев', inplace = True)
df1['Владение'].fillna('не имел владельцев', inplace = True)

In [65]:
df1 = df1.reset_index(drop=True)

In [66]:
df1[df1.mileage >400000]

,bodyType,color,fuelType,modelDate,numberOfDoors,productionDate,vehicleTransmission,engineDisplacement,enginePower,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,mileage
1965,седан,чёрный,бензин,1986.0,4.0,1988.0,автоматическая,3.0 LTR,188 N12,"['Регулировка передних сидений по высоте', 'Лю...",задний,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,не имел владельцев,105000.0,403890
2771,седан,серебристый,бензин,1987.0,4.0,1993.0,механическая,2.5 LTR,192 N12,"['Bluetooth', 'USB', 'Мультимедиа система с ЖК...",задний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,1 год и 1 месяц,180000.0,559233


In [67]:
df1 = df1.drop(df1[df1.mileage >400000].index)

In [69]:
df1['Привод'].value_counts()

полный      4672
задний      2703
передний      21
Name: Привод, dtype: int64

In [70]:
df1['fuelType'].value_counts()

бензин     4766
дизель     2619
гибрид        9
электро       2
Name: fuelType, dtype: int64

In [71]:
df1 = df1.drop(df1[df1['Привод'] == 'передний'].index)
df1 = df1.drop(df1[(df1['fuelType'] == 'гибрид') | (df1['fuelType'] == 'электро')].index)

In [72]:
df.drop(['Руль','Владение','Состояние'], axis = 1, inplace = True)
df1.drop(['Руль','Владение','Состояние'], axis = 1, inplace = True)

In [74]:
df['modelDate'] = 2020 - df['modelDate']
df1['modelDate'] = 2020 - df1['modelDate']
df['productionDate'] = 2020- df['productionDate']
df1['productionDate'] = 2020- df1['productionDate']

In [76]:
y_real = pd.read_csv('sample_submission.csv')
sample_submission = y_real.copy()

In [77]:
def del_first_space(x):
    a = []
    for row in x:
        if row[0] == ' ':
            row = row[1:]
        a.append(row)
    return a

In [78]:
df1['Комплектация'].fillna('Комплектация отсутствует', inplace = True)
df1['Компл'] = df1['Комплектация'].apply(lambda x: x.replace('[', '').replace(']', '').replace("'", ''))
df1['Компл'] = df1['Компл'].apply(lambda x: x.split(','))
df1['Компл'] = df1['Компл'].apply(del_first_space)
df1.drop(['Комплектация'], axis = 1, inplace = True)

In [79]:
def del_space(x):
    a = []
    for row in x:
        if row != '':
            a.append(row)
    return a

In [80]:
df['Комплектация'].fillna('Комплектация отсутствует', inplace = True)
df['Компл'] = df['Комплектация'].apply(lambda x: x.replace('[', '').replace(']', '').replace("'", '').replace("name", '').replace("values", '')
                                       .replace('"', '').replace("{", '').replace("}", '').replace(":", '').replace('Обзор', '')
                                       .replace('Элементы экстерьера', '').replace('Защита от угона', '').replace('Мультимедиа', '')
                                       .replace('Салон', '').replace('Комфорт', '').replace('Безопасность', '').replace('Прочее', ''))
df['Компл'] = df['Компл'].apply(lambda x: x.split(','))
df['Компл'] = df['Компл'].apply(del_space)
df.drop(['Комплектация'], axis = 1, inplace = True)
df['Компл'] = df['Компл'].apply(del_first_space)

In [81]:
df['price'] = 0
df['sample'] = 0
df1['sample'] = 1
data = pd.concat([df, df1], sort=False, ignore_index = True)

In [82]:
data2 = data['Компл']

In [83]:
data2 = pd.get_dummies(data2.apply(pd.Series).stack(), dummy_na=True).sum(level=0)

In [84]:
data = pd.get_dummies(data, columns = ['bodyType', 'color', 'fuelType', 'vehicleTransmission', 'enginePower',
                                           'Привод', 'Владельцы', 'ПТС', 'engineDisplacement', 'Таможня',], dummy_na=True)

In [85]:
data = pd.concat([data, data2], axis = 1)

In [86]:
data.drop(['Компл'], axis = 1, inplace = True)

In [87]:
train_data = data.query('sample==1').drop('sample', axis = 1)
test_data = data.query('sample==0').drop(['sample', 'price'], axis = 1)

In [88]:
X = train_data.drop(['price'], axis = 1)
y = train_data['price']
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [89]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, ExtraTreesRegressor, BaggingRegressor

In [97]:
test_data.fillna(0, inplace = True)

In [98]:
models = [RandomForestRegressor(n_estimators =250,random_state = 42, n_jobs = -1, verbose = 1),
         BaggingRegressor(ExtraTreesRegressor(random_state=42), random_state=42)]

def stacking_model_predict(models, X, y, test_data, sample_submission):
    for model_ in tqdm(models):
        model_.fit(X, y)
        pred_subm = model_.predict(test_data)
        sample_submission[str(model_)[:6]] = pred_subm
        #sample_submission[str(model_)[:6]] = sample_submission[str(model_)[:6]]
        #sample_submission[str(model_)[:6]] = sample_submission[str(model_)[:6]]
    sample_submission['price'] = sample_submission.iloc[:,2:].mean(axis=1)
    sample_submission[['id', 'price']].to_csv(f'submission_v2.csv', index=False)
    sample_submission.head(10)

stacking_model_predict(models, X, y, test_data, y_real)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    8.8s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:    0.0s finished


In [99]:
sample_submission = pd.read_csv('submission_v2.csv')

In [100]:
sample_submission['price'] = sample_submission['price'].apply(lambda x: round(x))

In [101]:
sample_submission

,id,price
0,0,1796184
1,1,2521419
2,2,1304659
3,3,2294731
4,4,4166240
...,...,...
3832,3832,1260032
3833,3833,2672915
3834,3834,581447
3835,3835,988476


In [102]:
sample_submission.to_csv(f'submission_v2.csv', index=False)